**Chapter 5 – Entrainer et mesurer la performance d'un Arbre de décision**

Étude de cas : Fraude aux assurances automobiles.

In [1]:
#importatation de modules commun

import numpy as np
import pandas as pd

# fixer le seed à une valeur constante pour la génération des mêmes indices alétaoire et donc avoir le même échantillon.
np.random.seed(4)

# pour la visualisation de figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
#lecture des deux datasets Train_FraudeAA et Test_FraudeAA
#le dataset original de 500 insatnces a été split en jeu d'entrainement 80%
# jeu de test (20%).
#Je l'ai aussi nettoyé : J'ai enlevé les trois variables ID. MARITAL STATUS ET INCOME
#A titre d'exemple, j'ai supprimé les outiliers (indvidus contenant des valeurs aberrantes)
#Il y en avait 45 individus parmi les 80%.
# le jeu de test est aussi nettoyé et mis dans la même confiuration que 
# le training set
data_train = pd.read_csv("Train_FraudeAA.CSV")
data_test = pd.read_csv("Test_FraudeAA.CSV")

In [3]:
data_train.head()

,NumClaimants,InjuryType,OvernightHospitalStay,ClaimAmount,TotalClaimed,NumClaims,NumSoftTissue,RatioSoftTissue,ClaimAmountReceived,Fraude
0,3,'Soft Tissue',No,638.0,5742,3,3.0,1.0,0,1
1,1,'Broken Limb',Yes,15876.0,18989,1,1.0,1.0,14156,0
2,1,'Soft Tissue',No,3780.0,7239,1,0.0,0.0,3780,0
3,1,Back,No,5711.0,4248,1,1.0,1.0,5711,0
4,3,'Soft Tissue',No,74365.0,0,0,0.0,0.0,0,1


In [4]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 364 entries, 0 to 363
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   NumClaimants           364 non-null    int64  
 1   InjuryType             364 non-null    object 
 2   OvernightHospitalStay  364 non-null    object 
 3   ClaimAmount            364 non-null    float64
 4   TotalClaimed           364 non-null    int64  
 5   NumClaims              364 non-null    int64  
 6   NumSoftTissue          364 non-null    float64
 7   RatioSoftTissue        364 non-null    float64
 8   ClaimAmountReceived    364 non-null    int64  
 9   Fraude                 364 non-null    int64  
dtypes: float64(3), int64(5), object(2)
memory usage: 28.6+ KB


In [5]:
data_train.shape

(364, 10)

In [6]:
data_test.shape

(92, 10)

In [7]:
data_test.head()

,NumClaimants,InjuryType,OvernightHospitalStay,ClaimAmount,TotalClaimed,NumClaims,NumSoftTissue,RatioSoftTissue,ClaimAmountReceived,Fraude
0,2,'Broken Limb',Yes,4824.0,16105,1,0.0,0.00,4824,0
1,3,Serious,Yes,47508.0,6136,1,1.0,1.00,47508,0
2,3,Back,Yes,72412.0,0,0,0.0,0.00,72412,0
3,3,'Soft Tissue',No,5403.0,0,0,0.0,0.00,5403,0
4,1,Back,No,3487.0,36306,4,1.0,0.25,3487,1


Transformation des variables qualitatives (object) en type vecteur nombre. 
Exigence de scikit-learn

Au lieu de passer par l'encoder OnehotEncoder, voici une approche plus simple dans ce contexte de l'exemple.
C'est d'utiliser la fonction get_dummy() de pandas qui produit le même résultat que l'encoder OneHotEncoder..

Nous avons 2 variables qualitatives:
1. InjuryType = BrokenLimb,Soft Tissue,Back,Serious
2. OvernightHospitalStay=yes/no




In [8]:
nouv_df_train=pd.get_dummies(data_train[data_train.columns[:-1]])

In [9]:
#voici la transformation
nouv_df_train.head()

,NumClaimants,ClaimAmount,TotalClaimed,NumClaims,NumSoftTissue,RatioSoftTissue,ClaimAmountReceived,InjuryType_'Broken Limb',InjuryType_'Soft Tissue',InjuryType_Back,InjuryType_Serious,OvernightHospitalStay_No,OvernightHospitalStay_Yes
0,3,638.0,5742,3,3.0,1.0,0,False,True,False,False,True,False
1,1,15876.0,18989,1,1.0,1.0,14156,True,False,False,False,False,True
2,1,3780.0,7239,1,0.0,0.0,3780,False,True,False,False,True,False
3,1,5711.0,4248,1,1.0,1.0,5711,False,False,True,False,True,False
4,3,74365.0,0,0,0.0,0.0,0,False,True,False,False,True,False


In [10]:
#Nous allons compléter le dataFrame nouv_df avec la variable cible dont le nom
#doit être class
nouv_df_train['class'] = data_train['Fraude']

In [11]:
nouv_df_train.head()

,NumClaimants,ClaimAmount,TotalClaimed,NumClaims,NumSoftTissue,RatioSoftTissue,ClaimAmountReceived,InjuryType_'Broken Limb',InjuryType_'Soft Tissue',InjuryType_Back,InjuryType_Serious,OvernightHospitalStay_No,OvernightHospitalStay_Yes,class
0,3,638.0,5742,3,3.0,1.0,0,False,True,False,False,True,False,1
1,1,15876.0,18989,1,1.0,1.0,14156,True,False,False,False,False,True,0
2,1,3780.0,7239,1,0.0,0.0,3780,False,True,False,False,True,False,0
3,1,5711.0,4248,1,1.0,1.0,5711,False,False,True,False,True,False,0
4,3,74365.0,0,0,0.0,0.0,0,False,True,False,False,True,False,1


In [12]:
# Séparer les données des variables prédictives (X) de la variable cible (Y)
X_train=nouv_df_train.iloc[:,:-1]
y_train=nouv_df_train['class']

# Entrainement par la validation croisée

In [14]:
#Maintenant, nous pouvons entrainer  pour constuire un classeur : clf
#en utilisant le nouveau dataframe modifié.: X_train.
#nous optons pour k=3, nombre de strates (blocks): voir l'option cv=3
#Ainsi on obtient 3 modèles avec 3 scores d'exactitude.


from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
clf= DecisionTreeClassifier(criterion="entropy")

cross_val_score(clf, X_train, y_train, cv=3, scoring="accuracy")



array([1., 1., 1.])

In [15]:
#Nous avons une performance d'exactitude moyenne de l'ordre de 1.
#Avant de conclure nous devons mesurer d'autres métriques.

In [17]:
#nous allons d'abord récupérer les résultats des prédictions 
#de la validation croisée aulieu des scores

from sklearn.model_selection import cross_val_predict
y_pred = cross_val_predict(clf, X_train, y_train, cv=3)


In [18]:
from sklearn import metrics
print(metrics.confusion_matrix(y_train,y_pred))


[[242   0]
 [  0 122]]


In [45]:
# ce qui confirme le résultat précédent. 
#FP=0 et FN=0. Donc la performance de 1. La classe positive est Fraude
#il ya 122 observations de la classe Fraude et 242 non-fraude 
#du jeu d'entrainement de 364

In [46]:
#mesure d'autres métriques à titre d'illustration: précsion et la rappel 

In [47]:
from sklearn.metrics import precision_score, recall_score, f1_score
precision_score(y_train, y_pred)

1.0

In [48]:
#ce qu'on peut aussi confirmer par : TP/TP+FP
#sachant que la classe positive est Fraude

In [49]:
122/(122+0)

1.0

In [50]:
#le rappel est de l'ordre de 
recall_score(y_train, y_pred)

1.0

In [51]:
#selon la formule TP/TP+FN
122/(0+122)

1.0

In [52]:
#et finalement la f1-mesure
f1_score(y_train, y_pred)

1.0

# Recherche du meilleur modèle par le réglage des hyper-paramètres

Bien que nous avons une très bonne performance avec la validation. Nous allons à titre d'illustration utiliser la recherche par 
quadrillage (GridSearch) pour l'optimisation des hyper-parametres: 
- Les hyper-paramètres: profondeur de l'arbre, le minimum d'instances qu'il faut avoir pour diviser un noeud.
- Ces deux hyper-paramètres ont un impact sur la grosseur de l'arbre et donc le sur-apprentissage.

In [19]:
#la profondeur de l'arbre est fixé dans une plage de valeurs entre 2-19 et donc 18 valeurs
#le minimum d'instances est une rangée de 3 valeurs possibles : 10, 15,, 20
# le nombre de combinaison de valeurs des hyper-paramètres : 18x3=54 combinaisons
# pour chaque combinaison, nous devoirs faire une validation croisée (cv=3) et donc entrainer 3 modèles.
#Par conséquent, le nombre total de modèle est 54X3=162 modèles.
# de ces 162 modèle entrainés, on prendra un seul, le meilleur !

from sklearn.model_selection import GridSearchCV
params = {'max_depth': list(range(2, 20)), 'min_samples_split': [10, 15, 20]}

grid_search_cv = GridSearchCV(DecisionTreeClassifier(criterion='entropy'), 
                              params, n_jobs=-1, verbose=1, cv=3)
grid_search_cv.fit(X_train, y_train)

Fitting 3 folds for each of 54 candidates, totalling 162 fits


GridSearchCV(cv=3, estimator=DecisionTreeClassifier(criterion='entropy'),
             n_jobs=-1,
             param_grid={'max_depth': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,
                                       14, 15, 16, 17, 18, 19],
                         'min_samples_split': [10, 15, 20]},
             verbose=1)

In [20]:
#le meilleur estimateur (modele) trouvé est:
grid_search_cv.best_estimator_

DecisionTreeClassifier(criterion='entropy', max_depth=2, min_samples_split=10)

## Validation avec un jeu de test

Validation du meilleur classeur clf avec le jeu de test.
Il faut s'assurer que le jeu de test est dans la même configuration
que le jeu d'entrainement.
Étant donné que nous avons transformé les variables qualitatives en en nombre,
Dans le jeu d'entrainement:
1.InjuryType = BrokenLimb,Soft Tissue,Back,Serious
2.OvernightHospitalStay=yes/no
Il faut faire la même chose dans le cas de jeu de test:


In [21]:
nouv_df_test=pd.get_dummies(data_test[data_test.columns[:-1]])

In [23]:
#il faut aussi renommer la variable cible
# et comme la variable cible est nommée Fraude, il faut aussi la renommer en class
nouv_df_test['class'] = data_test['Fraude']

In [24]:
#on peut calculer les frequences des observations par classe 
# 1. Fraude et 0 Non-Fraude
nouv_df_test["class"].value_counts()

class
0    56
1    36
Name: count, dtype: int64

In [25]:
#on peut calculer les frequences relative des observations par classe 
# 1. Fraude et 0 Non-Fraude
nouv_df_test["class"].value_counts(normalize=True)

class
0    0.608696
1    0.391304
Name: proportion, dtype: float64

In [26]:
#séparons les donnees de test des variables prédictives de celles 
#de la variable cible

X_test=nouv_df_test.iloc[:,:-1]
y_test=nouv_df_test['class']

In [27]:
from sklearn.metrics import accuracy_score

y_pred = grid_search_cv.predict(X_test)
accuracy_score(y_test, y_pred)

#Voici le résultat de la performance qui est de l'ordre:

0.9891304347826086

In [28]:
#mesure des métriques
print(metrics.confusion_matrix(y_test,y_pred))

[[56  0]
 [ 1 35]]


In [29]:
#il y a 1 Faux négatif !

In [30]:
#Nous pouvons en déduire les indicateurs de performance usuels
#taux d'exactitude (succes) ou l'accuracy de la matrice de confusion
#: (56+35)/(56+35+1)
print(metrics.accuracy_score(y_test,y_pred))

0.9891304347826086


In [31]:
#A partir de la nous pouvons dériver le taux d’erreur
print(1.0-metrics.accuracy_score(y_test,y_pred))

0.010869565217391353


In [32]:
# Si la classe Fraude est la classe positive (modalité cible) que l'on vise à identifier en priorité
#l'option pos_label est utilisée pour indique la catégorie. Par exemple la Fraude
#class=1
#le rappel de cette classe est donnée par: (35+1)/36
#
print(metrics.recall_score(y_test, y_pred, pos_label=1))
# c'est l'équivalent de TVP pour les fraudes qui est de l'ordre de:

0.9722222222222222


In [33]:
#et la précision pour les fraudes sera de l'ordre de (35/0+35) TP/TP+FP:
#C'est en lien avec le TFP : FP/TP+FP (0%)pour la fraude.
#la précison 1-TFP = 1.
print(metrics.precision_score(y_test, y_pred, pos_label=1))

1.0


In [67]:
#le F1-Score qui est une moyenne harmonique entre rappel et précision
#pour la classe Fraude
print(metrics.f1_score(y_test,y_pred,pos_label=1))

0.9859154929577464


Scikit-Learn propose un rapport global intégrant ces différents éléments avec la fonction classification_report() :

In [34]:
print(metrics.classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99        56
           1       1.00      0.97      0.99        36

    accuracy                           0.99        92
   macro avg       0.99      0.99      0.99        92
weighted avg       0.99      0.99      0.99        92

